# Belegaufgabe zum Blockkurs Python
## Simulation mit vtk
Ziel ist es ein Modul zu schreiben, welches eine Windmühle simuliert. Die Windmühle soll dabei nur
aus primitiven Objekten zusammengesetzt werden und einen Rotor besitzen der sich während der
Simulation dreht. Außerdem soll eine kleine GUI geschrieben werden (mit PyQt), worüber sich die
Simulation starten lässt und sich entsprechende Parameter (Rotationsgeschwindigkeit, Höhe, etc.)
einstellen lassen! Genutzt werden sollen die in der Übung vorgestellten vtk-Objektwrapperklassen!

------
The aim is to write a module that simulates a windmill. The windmill should only be made up of primitive objects and have a rotor that rotates during the simulation. A small GUI should also be written (with PyQt) through which the simulation can be started and the relevant parameters (rotation speed, height, etc.) can be set! The vtk object wrapper classes presented in the exercise should be used!



# Todo
- [X] class for objects
- [X] assemle windmill
- [x] vtk ???
- [ ] add GUI
- [ ] add dynamics
- [ ] add buttons



In [57]:

material_dict = {}

def add_material(name: str, material_properties: tuple):
    if name not in material_dict:
        material_dict[name] = material_properties
    else: 
        raise Exception("Such material already exists")
    
def get_material_list():
    return material_dict.keys()


class Structure:
    def __init__(self, name: str, mass: float, material: str, *dimensions: float):
        self._name = name
        self._mass = mass
        if material not in material_dict:
            raise Exception("No such material known")
        else:
            self._material = material
        self._dim = []
        if len(dimensions) != 2:
            raise ValueError("Need exactly 2 dimensions")
        else:
            self._dim = sorted(dimensions)

    def get_dimensions(self):
        return self._dim

    def get_material(self):
        return self._material

    def __str__(self):
        return "This is a %s %s of %d kg mass, made of %s. The size is %dx%d" % (self.__class__.__name__, self._name, self._mass, self._material, self._dim[0], self._dim[1])

structure_list = []

class Rotor(Structure):
    def __init__(self,  name: str, mass: float, material: str, max_rpm: float, model: str,  efficiency: float, *dimensions: float):
        super().__init__(name, mass, material, *dimensions)
        self._max_rpm = max_rpm
        self._model = model
        self._efficiency = efficiency

rotor_list = []

class Wing(Structure):
        def __init__(self,  name: str, mass: float, material: str, number_of_wings: int, *dimensions: float):
            super().__init__(name, mass, material, *dimensions)
            self._now = number_of_wings

wing_list = []



There are objects:

- Fundament (name, mass, width, length, material)
- Tube (name, mass, diameter, heigth, material)
- Rotor (name, mass, max_rpm, width=length, height, efficiency, model)
- Wing (name, mass, number_of_wings, material, length, width)
---
therefore 
- class Structure(name, mass, dimensions, material) can be used for everything ->
- Rotor is an ingeritage of sructure with max_rpm, model and efficiency parameter
- Wing is an ingeritage of structure with number_of_wings parameter

In [58]:
add_material("Steel", ((227, 213, 215), 600, 90))  # colour, how strong it is, how it heats
add_material("Wood", ((102, 46, 16), 100, 30))
add_material("Glass", ((209, 243, 255), 20, 40))
add_material("Concrete", ((124, 135, 135), 200, 20))

print(get_material_list())


Fundament1 = Structure("Fundament", 1000.0, "Concrete", 40, 40)
structure_list.append(Fundament1)

print(Fundament1)
    
Rotor1 = Rotor("Rotor", 420, "Steel", 1400, "mk. 1", 0.6, 8, 6)   
rotor_list.append(Rotor1)

print(Rotor1) 

Tube1 = Structure("Tube", 650.0, "Glass", 4, 60)
structure_list.append(Tube1)

print(Tube1)

Wing1 = Wing("Wing 3x", 80, "Wood", 3, 30, 4)
wing_list.append(Wing1)

print(Wing1)

dict_keys(['Steel', 'Wood', 'Glass', 'Concrete'])
This is a Structure Fundament of 1000 kg mass, made of Concrete. The size is 40x40
This is a Rotor Rotor of 420 kg mass, made of Steel. The size is 6x8
This is a Structure Tube of 650 kg mass, made of Glass. The size is 4x60
This is a Wing Wing 3x of 80 kg mass, made of Wood. The size is 4x30


In [51]:


def main():
    colors = vtkNamedColors()

    # Create a cylinder
    cylinderSource = vtkCylinderSource()
    cylinderSource.SetCenter(0.0, 0.0, 0.0)
    cylinderSource.SetRadius(5.0)
    cylinderSource.SetHeight(7.0)
    cylinderSource.SetResolution(100)

    # Create a mapper and actor
    mapper = vtkPolyDataMapper()
    mapper.SetInputConnection(cylinderSource.GetOutputPort())
    actor = vtkActor()
    actor.GetProperty().SetColor(colors.GetColor3d('Cornsilk'))
    actor.SetMapper(mapper)

    # Create a renderer, render window, and interactor
    renderer = vtkRenderer()
    renderWindow = vtkRenderWindow()
    renderWindow.SetWindowName('Cylinder')
    renderWindow.AddRenderer(renderer)
    renderWindowInteractor = vtkRenderWindowInteractor()
    renderWindowInteractor.SetRenderWindow(renderWindow)

    # Add the actor to the scene
    renderer.AddActor(actor)
    renderer.SetBackground(colors.GetColor3d('DarkGreen'))

    # Render and interact
    renderWindow.Render()
    renderWindowInteractor.Start()


#if __name__ == '__main__':
#    main()

Coordiatetes are always ```x, y, z```, where x-y in horisontal and z is vertical for normal state, therefore:
- For Fundament coordinates of the center should be ```0, 0, -0.05*tube._dim[1]``` and dimensions respectively ```dim[1]//2, dim[0]//2, 0.1 * tube._dim[1]```: W, L predefined, H = .1 of tube's H.
- Then the top surface of fundament shoud be ```z=0``` and it will be centered over (0, 0) (x,y)-wise
-----
- For Tube coordinates of the center are ```(0, 0, tube._dim[1]*0.5)```. Radius is tube._dim[0]*0.5 and Height = tube._dim[1]
-----
- For Rotor coordinates of the center are ```(0, 0, tube._dim[1] + rotor.dim[0]//2)``` and dimensions rotor.dim[1]x rotor.dim[1] x rotor.dim[0] (it is square 10 x 10 and 8 in height for example)
-----
- For Fan (which is wings * number of wings) center is ```(0, rotor.dim[1]//2,  tube._dim[1] + rotor.dim[0]//2 + wing.dim[0]//2)``` -- z is the same, x is also 0, y is moved by a half thickness of rotor and a half thickness of wing. Dimensions No idea

In [65]:
import numpy as np

class Windmill():
    def __init__(self, fund: Structure, tube: Structure, rotor: Rotor, wing: Wing):
        self._fund = fund
        self._tube = tube
        self._wing = wing
        self._rotor = rotor
        self.vtk_parts_list = []
        self.vtk_actors_list = []
        self.fan_position = np.array([0,0,0])
        self.fan_orientation = np.eye(3)
    

    def create_vtkCubeSource(self, struct, center, l, w, h):
        vtk_obj = vtkCubeSource()
        vtk_obj.SetCenter(center)
        vtk_obj.SetXLength(l)
        vtk_obj.SetYLength(h)
        vtk_obj.SetZLength(w)
        vtk_obj.structure = struct

        return vtk_obj
    

    # rotation axis is (0, 0, 1)
    #rotation matrix around this axis for phi rad is
    ''' 
    (cos phi + (1-cos phi) x*x) ((1-cos phi)*(xy)-sin phi z) ((1-cos phi)xz + sin phi y)
    ((1-cos phi)(yx)+sin phi z) (cos phi + (1-cos phi) y*y)) ((1-cos phi)yz + sin phi x)
    (1-cos phi)zx-sin phi y     1-cos phi zy + sin phi x       cos phi + (1-cos phi)z*z
    '''
    def turn_by(self, pos, ori, phi):

        rotation_matrix = np.matrix([[np.cos(phi), -np.sin(phi), 0],
                                     [np.sin(phi), np.cos(phi), 0],
                                     [0, 0, 1]])

        pos_new = pos * rotation_matrix       
        ori_new = ori
        return pos_new, ori_new


    def create_vtkCylinderSource(self, struct, center, r, h, resolution = 100):
        vtk_obj = vtkCylinderSource()
        vtk_obj.SetCenter(center)
        vtk_obj.SetRadius(r/2)
        vtk_obj.SetHeight(h)
        vtk_obj.SetResolution(resolution)
        vtk_obj.structure = struct

        return vtk_obj


    def assemble(self):
        tube_d, tube_h = self._tube.get_dimensions()
        fund_w, fund_l = self._fund.get_dimensions()  # W < L
        fund_h = tube_h * 0.1 
        rotor_h, rotor_w = self._rotor.get_dimensions()  # L is x, W is z, H is y, H < W = L
        rotor_l = rotor_w
        wing_w, wing_l = self._wing.get_dimensions()
        wing_h = wing_w * 0.1
        
        self._fund_center = (0.0, -0.5 * fund_h, 0.0)
        self._tube_center = (0.0, tube_h * 0.5, 0.0)
        self._rotor_center = (0.0, tube_h + rotor_h * 0.5, 0.0)
        self._fan_center = (wing_l * 0.5, tube_h + rotor_h * 0.5, rotor_w * 0.5 + wing_h * 0.5)
        
        print("fund center " + str(self._fund_center))
        print("fund l w h %f, %f, %f" % (fund_l, fund_w, fund_h))
        print("tube center " + str(self._tube_center))
        print("tube r, h %f, %f" %(tube_d/2, tube_h))
        print('rotor center' + str(self._rotor_center))
        print('fan center' + str(self._fan_center))
        self.Fundament_vtkSource = self.create_vtkCubeSource(self._fund, self._fund_center, fund_l, fund_w, fund_h)
        self.vtk_parts_list.append(self.Fundament_vtkSource)
        self.Rotor_vtkSource = self.create_vtkCubeSource(self._rotor, self._rotor_center, rotor_l, rotor_w, rotor_h)
        self.vtk_parts_list.append(self.Rotor_vtkSource)
#NEED TO CHANGE
        self.Fan_vtkSource = self.create_vtkCubeSource(self._wing, self._fan_center, wing_l, wing_h, wing_w) 
        self.vtk_parts_list.append(self.Fan_vtkSource)
#NEED TO CHANGE
        self.Tube_vtkSource = self.create_vtkCylinderSource(self._tube, self._tube_center, tube_d, tube_h, 10)
        self.vtk_parts_list.append(self.Tube_vtkSource)

        return self.vtk_parts_list
    

    def create_vtkActor(self, vtk_obj):
        mapper = vtkPolyDataMapper()
        mapper.SetInputConnection(vtk_obj.GetOutputPort())
        mapper.ScalarVisibilityOff()
        actor = vtkActor()
        actor.SetMapper(mapper)

        material = vtk_obj.structure.get_material()
        color = tuple(i/255 for i in material_dict[material][0])  # make 0-255 to 0.0-1.0

        actor.GetProperty().EdgeVisibilityOn()
        actor.GetProperty().SetLineWidth(2)

        
        actor.GetProperty().SetColor(color)
        actor.GetProperty().Modified()
        
        print(material + str(actor.GetProperty().GetColor()))
        
        return actor
    

    def initiate_all_actors(self):
        self.vtk_actors_list = []
        for vtk_part in self.vtk_parts_list:
            actor = self.create_vtkActor(vtk_part)
            self.vtk_actors_list.append(actor)

        return self.vtk_actors_list


    def render(self, lst):
        # Create a renderer, render window, and interactor
        renderer = vtkRenderer()
        renderWindow = vtkRenderWindow()
        renderWindow.AddRenderer(renderer)
        renderWindowInteractor = vtkRenderWindowInteractor()
        renderWindowInteractor.SetRenderWindow(renderWindow)
                
        for actor in lst:
            renderer.AddActor(actor)
                
        renderer.SetBackground(.8,.8,.8)
        renderWindow.Render()
        renderWindowInteractor.Start()
        
        pass

    def change_rotation_speed(self):
        pass

    def change_part(self):
        pass


if __name__ is '__main__':
    Windmill1 = Windmill(Fundament1, Tube1, Rotor1, Wing1)

    Windmill1.assemble()


    list_of_actors = Windmill1.initiate_all_actors()
    Windmill1.render(list_of_actors)


<>:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\user\AppData\Local\Temp\ipykernel_15736\455553112.py:122: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if __name__ is '__main__':


fund center (0.0, -3.0, 0.0)
fund l w h 40.000000, 40.000000, 6.000000
tube center (0.0, 30.0, 0.0)
tube r, h 2.000000, 60.000000
rotor center(0.0, 63.0, 0.0)
fan center(15.0, 63.0, 4.2)
Concrete(0.48627450980392156, 0.5294117647058824, 0.5294117647058824)
Steel(0.8901960784313725, 0.8352941176470589, 0.8431372549019608)
Wood(0.4, 0.1803921568627451, 0.06274509803921569)
Glass(0.8196078431372549, 0.9529411764705882, 1.0)


In [ ]:
import math
import numpy as np
'''
[t00 t01 t02 x]
[t10 t11 t12 y]
[t20 t21 t22 z]
[0   0   0   0]

t is 3x3 matrix of the orientation

x y z are the position

we move wing in rotate wing 
'''
